<a href="https://colab.research.google.com/github/sushantsp/Hackathons/blob/master/Airplane_Crash_Severity_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

!ls '/content/drive/My Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'   Dataset  'Getting started.pdf'


In [0]:
import pandas as pd


train = pd.read_csv('/content/drive/My Drive/Dataset/train.csv')
test = pd.read_csv('/content/drive/My Drive/Dataset/test.csv')

train.columns
train.dtypes
# ALL features are either float or integers. and the output is categorical into 4 values. which will be mapped to 1,2,3,4
train.Violations.value_counts() # this is categorical Variable
train.Accident_Type_Code.value_counts() # This is a Categorical variable


train = pd.get_dummies(train, columns =['Accident_Type_Code','Violations'])
test = pd.get_dummies(test, columns =['Accident_Type_Code','Violations'])

train.columns
# train = train[['Accident_ID', 'Safety_Score', 'Days_Since_Inspection',
#        'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces',
#        'Cabin_Temperature', 'Max_Elevation', 'Adverse_Weather_Metric',
#        'Accident_Type_Code_1', 'Accident_Type_Code_2',
#        'Accident_Type_Code_3', 'Accident_Type_Code_4', 'Accident_Type_Code_5',
#        'Accident_Type_Code_6', 'Accident_Type_Code_7', 'Violations_0',
#        'Violations_1', 'Violations_2', 'Violations_3', 'Violations_4',
#        'Violations_5','Severity']]
train = train[['Accident_ID', 'Safety_Score', 'Days_Since_Inspection', 
       'Control_Metric','Severity']]
# 'Severity' is sent to last columns as that is the output. Accident_ID is made first columns since thats the output format.

train.Severity.value_counts().sum() # 10000 values
train.shape # 10000 x 23
# to confirm the values in the output and shape of the train data.
train.Severity.value_counts() # Distribution of the data along output types.

# Mapping all 4 types to numeric values. 
train.Severity = train.Severity.map({'Highly_Fatal_And_Damaging':0,
                                     'Significant_Damage_And_Serious_Injuries':1,
                                     'Minor_Damage_And_Injuries':2,
                                     'Significant_Damage_And_Fatalities':3},)
test.columns
# Adding Accident_ID as the first column to the test data.
# test = test[['Accident_ID','Safety_Score', 'Days_Since_Inspection', 'Total_Safety_Complaints',
#        'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature',
#        'Max_Elevation', 'Adverse_Weather_Metric',
#        'Accident_Type_Code_1', 'Accident_Type_Code_2', 'Accident_Type_Code_3',
#        'Accident_Type_Code_4', 'Accident_Type_Code_5', 'Accident_Type_Code_6',
#        'Accident_Type_Code_7', 'Violations_0', 'Violations_1', 'Violations_2',
#        'Violations_3', 'Violations_4', 'Violations_5']]
test = test[['Accident_ID','Safety_Score', 'Days_Since_Inspection','Control_Metric']]

train.shape, test.shape # one extra column in train data which is the output.
# since ID's are not the part of the data input, they are saved into different columns and later will mapped onto
# the output data for submission.
Acc_ID_Train = train.Accident_ID
Acc_ID_Test = test.Accident_ID
# Dropping the Accident ID
train = train.iloc[:,1:]
test = test.iloc[:,1:]
# train= train[(np.abs(stats.zscore(train)) < 4).all(axis=1)]

# Create X_train, y_train and we have test data withour output.
X = train.iloc[:,0:-1]
y = train.iloc[:,[-1]] # severityb

# X.shape, test.shape

In [0]:
from sklearn import datasets, linear_model, metrics 
import xgboost as xgb
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.05, random_state = 0)
# param = {
#     eta =  0.6, 
#     max_depth= 100,  
#     objective: 'multi:softprob',  
#     num_class: 4} 

xgb_model = xgb.XGBClassifier(n_estimators = 2000,eta =0.6, max_depth= 25,   min_child_weight = 3, objective = 'multi:softprob',  num_class = 4, random_state=42)
xgb_model.fit(X_train, Y_train)

y_pred = xgb_model.predict(X_test)


print(confusion_matrix(Y_test, y_pred))
# # Fitting XGBoost to the Training set
# D_train = xgb.DMatrix(X_train, label=Y_train)
# D_test = xgb.DMatrix(X_test, label=Y_test)


# param =  {'colsample_bytree': 0.7,
#  'eta': 0.05,
#  'gamma': 0.2,
#  'max_depth': 15,
#  'min_child_weight': 1,
#  'objective': 'multi:softprob','num_class': 4} 

# steps = 20  # The number of training iterations
# model = xgb.train(param, D_train, steps)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
cm

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[149   1   0   0]
 [  1 152   1   0]
 [  3   1 126   1]
 [  1   0   0  64]]


array([[149,   1,   0,   0],
       [  1, 152,   1,   0],
       [  3,   1, 126,   1],
       [  1,   0,   0,  64]])

In [0]:
metrics.accuracy_score(Y_test, y_pred)*100

97.8

In [0]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9644328127486417
Recall = 0.9659871807581731
Accuracy = 0.966


In [0]:
# Predicting the Test set results
y_pred = model.predict(D_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
cm

ValueError: ignored

In [0]:
xgb_model = xgb.XGBClassifier(n_estimators = 3000,eta =0.6, max_depth= 25, min_child_weight = 3, objective = 'multi:softprob',  num_class = 4, random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
y_pred  = xgb_model.predict(test)
# sub = pd.DataFrame(Acc_ID_Test)
# sub['Severity'] = y_pred.tolist()
# sub.Severity = sub.Severity.map({0:'Highly_Fatal_And_Damaging',
#                    1:'Significant_Damage_And_Serious_Injuries',
#                    2:'Minor_Damage_And_Injuries',
#                    3:'Significant_Damage_And_Fatalities'})
# sub.to_csv("/content/drive/My Drive/Dataset/RF_submission_XGB_080220.csv",index = None, header=True)

In [0]:
test_1 = test.copy()
test_1['Severity'] = y_pred.tolist()
test_1.head()

,Safety_Score,Days_Since_Inspection,Control_Metric,Severity
0,19.497717,16,72.151322,0
1,58.173516,15,64.585232,3
2,33.287671,15,64.721969,1
3,3.287671,21,66.362808,0
4,10.867580,18,56.107566,3


In [0]:
new = train.append(test_1)

# Create X_train, y_train and we have test data withour output.
X = new.iloc[:,0:-1]
y = new.iloc[:,[-1]] # severity

from sklearn import datasets, linear_model, metrics 
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 0)


xgb_model = xgb.XGBClassifier(n_estimators = 3000,eta =0.6, max_depth= 25,   min_child_weight = 3, objective = 'multi:softprob',  num_class = 4, random_state=42)
xgb_model.fit(X, y)



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=25,
              min_child_weight=3, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, num_class=4, objective='multi:softprob',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

In [0]:
sub = pd.DataFrame(Acc_ID_Test)
sub['Severity'] = y_pred.tolist()
sub.Severity = sub.Severity.map({0:'Highly_Fatal_And_Damaging',
                   1:'Significant_Damage_And_Serious_Injuries',
                   2:'Minor_Damage_And_Injuries',
                   3:'Significant_Damage_And_Fatalities'})
sub.to_csv("/content/drive/My Drive/Dataset/RF_submission_XGB_comb_080220.csv",index = None, header=True)

In [0]:
# from sklearn.model_selection import GridSearchCV

# clf = xgb.XGBClassifier()
# parameters = {
#      "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
#      "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
#      "min_child_weight" : [ 1, 3, 5, 7 ],
#      "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
#      "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
#      }

# grid = GridSearchCV(clf,
#                     parameters, n_jobs=4,
#                     scoring="neg_log_loss",
#                     cv=3)

# grid.fit(X_train, Y_train)

KeyboardInterrupt: ignored

In [0]:
grid.best_params_

{'colsample_bytree': 0.7,
 'eta': 0.05,
 'gamma': 0.2,
 'max_depth': 15,
 'min_child_weight': 1}

In [0]:
param = {'colsample_bytree': 0.7,
 'eta': 0.05,
 'gamma': 0.2,
 'max_depth': 15,
 'min_child_weight': 1}
num_round = 2
bst = xgb.train(param, D_train, num_round)
#
grid.predict(D_test)

TypeError: ignored

In [0]:
from sklearn import datasets, linear_model, metrics 
import xgboost as xgb
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.05, random_state = 0)


xgb_model = xgb.XGBClassifier(n_estimators = 1000, eta =0.6, max_depth= 100,  objective = 'multi:softprob',  num_class = 4, random_state=42)
xgb_model.fit(X, y)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=100,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, num_class=4, objective='multi:softprob',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

In [0]:
y_pred  = xgb_model.predict(test)
sub = pd.DataFrame(Acc_ID_Test)
sub['Severity'] = y_pred.tolist()
sub.Severity = sub.Severity.map({0:'Highly_Fatal_And_Damaging',
                   1:'Significant_Damage_And_Serious_Injuries',
                   2:'Minor_Damage_And_Injuries',
                   3:'Significant_Damage_And_Fatalities'})
sub.to_csv("/content/drive/My Drive/Dataset/RF_submission_XGB_070220.csv",index = None, header=True)

In [0]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [25,50,75],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4]
             }

numFolds = 5
kfold_5 = cross_validation.KFold(n = len(X), shuffle = True, n_folds = numFolds)

clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5, # you want 5 here not 25 if I understand you correctly 
                         scoring = 'roc_auc', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)